In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 31580
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = 'assets/logo.png'  # replace with the correct path to the image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Center(html.H1('Grazioso Salvare Dashboard')),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'})),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'all'},
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'wilderness'},
            {'label': 'Disaster/Tracking', 'value': 'disaster'}
        ],
        value='all',
        inline=True,
        style={'margin': '20px'}
    ),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         style_table={'overflowX': 'auto'},
                         filter_action='native',
                         sort_action='native',
                         page_action='native',
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'wilderness':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Bloodhound", "Bluetick Coonhound"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}

    filtered_data = pd.DataFrame.from_records(db.read(query))
    if '_id' in filtered_data.columns:
        filtered_data.drop(columns=['_id'], inplace=True)

    return filtered_data.to_dict('records')

@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    if viewData is None:
        return html.Div("No data available")

    dff = pd.DataFrame.from_dict(viewData)
    breed_chart = px.histogram(dff, x='breed', title='Breed Distribution')
    return dcc.Graph(figure=breed_chart)

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    if viewData is None or not viewData:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer(id="base-layer-id")])

    dff = pd.DataFrame.from_dict(viewData)
    if index is None or not index:
        row = 0
    else:
        row = index[0]

    map_markers = [
        dl.Marker(
            position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
            children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row]['name'])])
            ]
        )
    ] if 'location_lat' in dff.columns and 'location_long' in dff.columns else []

    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id")
    ] + map_markers)

app.run_server(debug=True)
